In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout

def load_data(filename, skiprows = 1):
    '''   
    Function loads data stored in the file filename and returns it as a numpy ndarray.\n",

    Inputs:
    filename: given as a string.
    Outputs:\n",
    Data contained in the file, returned as a numpy ndarray\n",
    '''
    return np.loadtxt(filename, skiprows=skiprows, delimiter=',')

Using TensorFlow backend.


In [3]:
train = load_data("data/train_2008.csv")
test = load_data("data/test_2012.csv")


In [4]:
import xgboost

from sklearn import cross_validation, tree, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

y_train_new = train[:,train.shape[1]-1:]
X_train = train[:,:train.shape[1]-1]
    
X_test = test[:,3:]
y_train_new = train[:,train.shape[1]-1:]
X_train = train[:,:train.shape[1]-1]
X_train = X_train[:,3:]


xgb.fit(X_train, y_train_new)



/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.08, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.75)

In [7]:

X_test = test[:,3:]
predictions = xgb.predict(X_test)

predictions = predictions.reshape(-1, 1)

print(predictions)
for i in predictions:
    if i[0] <0:
        i[0] = 0
    elif i[0]>1:
        i[0] = 1
        
print(min(predictions))

ids = test[:,:1]
ids = ids.astype(int)
predictions = predictions.astype(float)
test = np.concatenate((ids, predictions), axis=1)

[[-0.03432846]
 [ 0.10428217]
 [ 0.10118091]
 ...
 [ 0.12150231]
 [ 0.12071863]
 [ 0.15512234]]
[0.]


In [9]:
np.savetxt('2012_xgb.csv', test, delimiter=',', fmt='%1.10f', header = 'id,target')

In [10]:
import csv
xg = []
ids = []
with open('2012_xgb.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        xg.append(row[1])
        ids.append(row[0])
xgprediction = np.array(xg[1:])
xgprediction = xgprediction.astype(float)
xgprediction = np.reshape(xgprediction, (-1, 1))
ids = np.array(ids[1:])
ids = ids.astype(int)
ids = np.reshape(ids, (-1, 1))

In [11]:
nn = []
with open('best_nn_2012.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        nn.append(row[1])
nnprediction = np.array(nn[1:])
nnprediction = nnprediction.astype(float)
nnprediction = np.reshape(nnprediction, (-1, 1))


In [13]:
merger = (nnprediction + xgprediction)/2


In [14]:
merger

array([[0.05012614],
       [0.15135108],
       [0.13426292],
       ...,
       [0.13285747],
       [0.11405876],
       [0.1500388 ]])

In [15]:
test = np.concatenate((ids, merger), axis=1)
np.savetxt('merge_xg_best_nn_2012.csv', test, delimiter=',', fmt='%1.10f', header = 'id,target')